<a href="https://colab.research.google.com/github/mahalakshmijinadoss/pooling/blob/main/goldbecktwtrobertabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers[sentencepiece]==4.10.0 datasets --quiet


In [ ]:
torch.cuda.clear_cache

In [3]:
!python -c  "import transformers;print(transformers.__version__)"


4.10.0


In [4]:
!pip install -qq ipython-autotime

%load_ext autotime

time: 200 µs (started: 2022-02-10 03:44:41 +00:00)


In [5]:
import torch
torch.cuda.empty_cache()


time: 1.44 s (started: 2022-02-10 03:44:41 +00:00)


In [8]:
import numpy as np
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

time: 346 ms (started: 2022-02-10 03:45:00 +00:00)


In [9]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

time: 6.58 s (started: 2022-02-10 03:45:03 +00:00)


In [10]:
import pandas as pd
import numpy as np


time: 1.52 ms (started: 2022-02-10 03:45:13 +00:00)


In [11]:
from transformers import DataCollatorWithPadding


time: 130 ms (started: 2022-02-10 03:45:15 +00:00)


In [12]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL = "cardiffnlp/twitter-roberta-base-sep2021"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sep2021 were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sep2021 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_p

time: 3.05 s (started: 2022-02-10 03:53:22 +00:00)


In [14]:
df= pd.read_csv('/content/goldbeck.csv',sep=',')

time: 87.9 ms (started: 2022-02-10 03:45:57 +00:00)


In [15]:
df.shape


(20360, 4)

time: 14 ms (started: 2022-02-10 03:46:03 +00:00)


In [17]:
df.head()

,Unnamed: 0,ID,Tweet,label
0,0,1,@Grumpy_P_Sloth @deanesmay feminists argue for...,H
1,1,2,1948Army of England helped the Jews to occupy ...,H
2,2,3,Dutch Leader Says Europe to collapse In 6 Week...,H
3,3,4,RT @__DeLay: The next day the Romans and the J...,H
4,4,5,RT @Bakersman_Joe: When Hitler Invited The Jew...,H


time: 37.5 ms (started: 2022-02-10 03:46:12 +00:00)


In [44]:
df['label'].value_counts()

N    15075
H     5285
Name: label, dtype: int64

time: 15.6 ms (started: 2022-02-10 03:56:57 +00:00)


In [45]:
df.loc[(df.label == 'H'), 'label'] = 1
df.loc[(df.label == 'N'), 'label'] = 0


time: 13.1 ms (started: 2022-02-10 03:57:08 +00:00)


In [46]:
X=df.Tweet.values
y=df.label.values

time: 3.31 ms (started: 2022-02-10 03:57:22 +00:00)


In [47]:
from sklearn.model_selection import train_test_split


time: 11.7 ms (started: 2022-02-10 03:57:24 +00:00)


In [48]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1, random_state=1)

time: 18.3 ms (started: 2022-02-10 03:57:25 +00:00)


In [49]:
import re

time: 1.89 ms (started: 2022-02-10 03:57:27 +00:00)


In [50]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

time: 5.2 ms (started: 2022-02-10 03:57:29 +00:00)


In [51]:
train_texts= np.array([preprocess(text) for text in X_train])
dev_texts= np.array([preprocess(text) for text in X_dev])
num_labels=2


time: 353 ms (started: 2022-02-10 03:57:31 +00:00)


In [52]:
train_texts=train_texts.tolist()
dev_texts=dev_texts.tolist()

time: 68.8 ms (started: 2022-02-10 03:57:33 +00:00)


In [53]:
train_texts[0:20]

["RT @user Men who want to impress other men rape women - way to go fellas  Australia Navy cadet filmed rape 'to be accepted' http",
 'That would explain why the \\"nice\\" feminists won\'t tell the radfems to STFU. http',
 "The Jews don't like Farrakhan, so they call me Hitler. Well, that's a good name. Hitler was a very great man.",
 "The New Yorker: #Republicans' #Frankenstein #Monster. #Capitalism #Plutocracy #WhitePower #Fascism  http / /t.co /TG6oKS6ATS",
 'Who led the Jews across the semi permeable membrane?   Osmoses',
 '@user #RespectTheWhiteShoes  #whitepower',
 'RT @user Are you guys against all indigenous peoples self determination on their ancestral lands or just the jews? #AskHamas',
 '@user @user but there is no evidence to prove #gamergate did it',
 "Ben Howe's conservatism: feminist, PC, pro-Islam, open borders, and a shill for big business.  http",
 "@user Given that logic, I'm sure the Jews felt very empowered in 1942.",
 "Following this logic, I should be terrified 

time: 6.35 ms (started: 2022-02-10 03:57:35 +00:00)


In [54]:
train_encodings = tokenizer(train_texts,truncation=True,)
dev_encodings = tokenizer(dev_texts,truncation=True)

time: 1.98 s (started: 2022-02-10 03:57:38 +00:00)


In [55]:
import torch

class TaskDatasetB(torch.utils.data.Dataset):

  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 11.3 ms (started: 2022-02-10 03:57:42 +00:00)


In [56]:
train_dataset = TaskDatasetB(train_encodings,y_train)
dev_dataset = TaskDatasetB(dev_encodings,y_dev)

time: 2.74 ms (started: 2022-02-10 03:57:43 +00:00)


In [57]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


time: 2.7 ms (started: 2022-02-10 03:57:45 +00:00)


In [58]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'OGBV-gender-twtrobertabase-en-goldebeck',
    run_name = "twt-roberta-en-3epoch",
    evaluation_strategy = 'epoch',
    save_strategy='epoch',
    num_train_epochs = 3,
    load_best_model_at_end = True,
    push_to_hub='OGBV-gender-twtrobertabase-en-goldbeck'
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


time: 16.5 ms (started: 2022-02-10 03:57:47 +00:00)


In [59]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions  = np.argmax(logits,axis=-1)

  acc = accuracy_score(labels,predictions)
  precision = precision_score(labels,predictions,average='weighted')
  recall = recall_score(labels,predictions,average='weighted')
  f1 = f1_score(labels,predictions,average='weighted')
  
  return {
     'acc' : acc,
     'precision' : precision,
     'recall' : recall,
     'f1' : f1
  }

time: 10.8 ms (started: 2022-02-10 03:57:49 +00:00)


In [34]:
del model 


time: 1.41 ms (started: 2022-02-10 03:52:58 +00:00)


In [35]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-

In [60]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,

    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
/content/OGBV-gender-twtrobertabase-en-goldebeck is already a clone of https://huggingface.co/Maha/OGBV-gender-twtrobertabase-en-goldebeck. Make sure you pull the latest changes with `repo.git_pull()`.


time: 5.77 s (started: 2022-02-10 03:57:56 +00:00)


In [61]:
trainer.train()


***** Running training *****
  Num examples = 18324
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6873
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Acc,Precision,Recall,F1
1,0.595300,0.597468,0.732809,0.537010,0.732809,0.619814
2,0.573300,0.567671,0.732809,0.537010,0.732809,0.619814
3,0.550700,0.561251,0.752947,0.756114,0.752947,0.676769


***** Running Evaluation *****
  Num examples = 2036
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to OGBV-gender-twtrobertabase-en-goldebeck/checkpoint-2291
Configuration saved in OGBV-gender-twtrobertabase-en-goldebeck/checkpoint-2291/config.json
Model weights saved in OGBV-gender-twtrobertabase-en-goldebeck/checkpoint-2291/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2036
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(re

TrainOutput(global_step=6873, training_loss=0.5689310123527412, metrics={'train_runtime': 1695.4122, 'train_samples_per_second': 32.424, 'train_steps_per_second': 4.054, 'total_flos': 1327148607676800.0, 'train_loss': 0.5689310123527412, 'epoch': 3.0})

time: 28min 15s (started: 2022-02-10 03:58:04 +00:00)


In [62]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 2036
  Batch size = 8


{'epoch': 3.0,
 'eval_acc': 0.7529469548133595,
 'eval_f1': 0.6767694098208894,
 'eval_loss': 0.5612505674362183,
 'eval_precision': 0.7561143570253707,
 'eval_recall': 0.7529469548133595,
 'eval_runtime': 16.3558,
 'eval_samples_per_second': 124.481,
 'eval_steps_per_second': 15.591}

time: 16.4 s (started: 2022-02-10 04:26:22 +00:00)


In [ ]:
import torch
torch.cuda.empty_cache()

time: 277 ms (started: 2022-02-09 18:07:42 +00:00)


In [63]:
model_path = 'OGBV-gender-twtrobertabase-en-goldbeck'


time: 1.75 ms (started: 2022-02-10 04:26:46 +00:00)


In [64]:
model.push_to_hub('OGBV-gender-twtrobertabase-en-goldbeck')


/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Maha/OGBV-gender-twtrobertabase-en-goldbeck into local empty directory.
Configuration saved in OGBV-gender-twtrobertabase-en-goldbeck/config.json
Model weights saved in OGBV-gender-twtrobertabase-en-goldbeck/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.37k/476M [00:00<?, ?B/s]

To https://huggingface.co/Maha/OGBV-gender-twtrobertabase-en-goldbeck
   781a14d..2b35ee3  main -> main



'https://huggingface.co/Maha/OGBV-gender-twtrobertabase-en-goldbeck/commit/2b35ee39f5fa838c8ceb5cf0f47e78500e3b800c'

time: 8min 13s (started: 2022-02-10 04:26:48 +00:00)


In [65]:
tokenizer.push_to_hub('OGBV-gender-twtrobertabase-en-goldbeck')


tokenizer config file saved in OGBV-gender-twtrobertabase-en-goldbeck/tokenizer_config.json
Special tokens file saved in OGBV-gender-twtrobertabase-en-goldbeck/special_tokens_map.json
To https://huggingface.co/Maha/OGBV-gender-twtrobertabase-en-goldbeck
   2b35ee3..c9ff47d  main -> main



'https://huggingface.co/Maha/OGBV-gender-twtrobertabase-en-goldbeck/commit/c9ff47d00892366b5cbc8931c81304a88989f023'

time: 6.09 s (started: 2022-02-10 04:36:07 +00:00)
